In [9]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


spark = SparkSession.builder.master('local[2]').getOrCreate()
#spark.conf.set('spark.cores.max', '1')
#spark.conf.set('spark.driver.memory','1000M')

sc = spark.sparkContext

spark

In [10]:
!pip install psutil
from psutil import virtual_memory
print(f"Your runtime has {virtual_memory().total / 1e9:.1f} GiB of RAM, {spark.sparkContext.defaultParallelism}cores")

Your runtime has 8.2 GiB of RAM, 2cores


## Word count

In [25]:
libros = sc.textFile("data/books")  # cargamos los libros
print(libros.count())
print(libros.take(5))

279091
['The Project Gutenberg EBook of Peter Pan, by James M. Barrie', '', '  This eBook is for the use of anyone anywhere at no cost and with', 'almost no restrictions whatsoever.  You may copy it, give it away or', 're-use it under the terms of the Project Gutenberg License included']


In [32]:
tuplas = libros.flatMap(lambda x: x.split()).map(lambda x: (x, 1))  # MAP (flat bc .split() returns a list)
tuplas.cache()  # cache results for re-use

PythonRDD[152] at RDD at PythonRDD.scala:53

In [34]:
result = tuplas.reduceByKey(lambda x,y: x+y)  # SHUFFLE (`ByKey`) & REDUCE
result.coalesce(1).saveAsTextFile("output/wc")  # save as one file